
<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Three: Classifier training and performance assessment. </h2>	

In [51]:
import numpy as np
import pandas as pd
import sklearn.model_selection

In [165]:
features_df = np.load('features_df.npy')

In [174]:
from sklearn.utils import shuffle

def train_test_split(features_df):
    #Normalize the features
    #X = (features_df[:,:-1] - features_df[:,:-1].mean(axis=1)[:, np.newaxis])/features_df[:,:-1].std(axis=1)[:, np.newaxis]
    X = features_df[:, :-1]
    y = features_df[:,-1]
    X, y = shuffle(X, y, random_state=0)
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y, train_size = 0.7, test_size=0.3)
    return [X_train, X_test, y_train, y_test]
#Split the data into a training set, and test set 

def accuracy(pred, actual):
    return sum(pred == actual)/len(actual)
# Calculate the accuracy percentage of the predicted values

In [175]:
def k_fold_cv(model, X, y, k=10):
    training_acc = 0
    testing_acc = 0
    cpt = 0
    step = len(X)//k
    for i in range(0,k):
        if i==k-1:
            X_test = X[i*step::]
            y_test = y[i*step::]
            X_train = X[0:i*step]
            y_train = y[0:i*step]
        else:
            X_test = X[i*step:(i+1)*step]
            y_test = y[i*step:(i+1)*step]
            X_train = np.concatenate((X[0:i*step],X[(i+1)*step::]))
            y_train = np.concatenate((y[0:i*step],y[(i+1)*step::]))
        
        model.fit(X_train, y_train)
        model_train_acc = model.evaluate(X_train, y_train)
        training_acc += model_train_acc
        model_test_acc = model.evaluate(X_test, y_test)
        testing_acc += model_test_acc
        print('Training {} of {}: - Training accuracy: {} - Testing accuracy: {}'.format(i+1, k, model_train_acc, model_test_acc))
    print('\nAverage Training Accuracy: {} / Average Testing Accuracy: {}'.format(training_acc/k, testing_acc/k))
    return training_acc/k, testing_acc/k

<h3>  Train models using all of the following methods below. Be sure to drop the actual image column, and the encoding</h3>	Take note of the differences in accuracy, and methods.

### For the following models, we used 5-fold cross validation to tune the hyper-parameters
We could have used nested cross validation to avoid overfitting the test set when both selecting the hyper parameter and evaluating the model

Logistic Regression

In [176]:
X_train, X_test, y_train, y_test = train_test_split(features_df)

In [177]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(penalty='none', solver='lbfgs', multi_class='ovr')
logisticRegr.fit(X_train, y_train)
logisticRegr.score(X_test, y_test)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/anaconda3

0.31042128603104213

K-nearest Neighbors

In [178]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(X_train, y_train)
neigh.score(X_test, y_test)

0.15521064301552107

Classification Tree

In [179]:
from sklearn import tree
clf_tree = tree.DecisionTreeClassifier(max_depth = 10)
clf_tree = clf_tree.fit(X_train, y_train)
clf_tree.score(X_test, y_test)

0.22838137472283815

Random Forest

In [180]:
from sklearn.ensemble import RandomForestClassifier
clf_randomforest = RandomForestClassifier(max_depth=10, random_state=0)
clf_randomforest.fit(X_train, y_train)
clf_randomforest.score(X_test, y_test)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.30155210643015523

Support Vector Machine

In [181]:
from sklearn.svm import SVC
clf_svm = SVC(gamma='auto')
clf_svm.fit(X_train, y_train)
clf_svm.score(X_test, y_test)

0.21729490022172948

### Meta Classifier
We are going to use the logistic regression classifier as well as the random forest classifier.

In [89]:
new_x = np.concatenate((logisticRegr.predict(X_train).reshape(-1,1),clf_randomforest.predict(X_train).reshape(-1,1)), axis=1)


In [90]:
meta_classifier = RandomForestClassifier(max_depth=10, random_state=0)
meta_classifier.fit(new_x, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [91]:
new_x_test = np.concatenate((logisticRegr.predict(X_test).reshape(-1,1),clf_randomforest.predict(X_test).reshape(-1,1)), axis=1)
meta_classifier.score(new_x_test, y_test)

0.2771618625277162

### Final Prediction
The meta classifier was less accurate than the random forest classifier. Let's train the random forest classifier with all the training data.

In [110]:
X = features_df[:,:-1]
y = features_df[:,-1]
clf_randomforest.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [98]:
import glob
from skimage import io

In [102]:
testing_set = {'Pictures':[], 'Names':[]}
for image in glob.glob('20_Validation/*.jpg'):
        testing_set['Names'].append(image.split('/')[1])
        img = io.imread(image)
        testing_set['Pictures'].append(img)
testing_set = pd.DataFrame(testing_set)

In [103]:
testing_set.head()

,Pictures,Names
0,"[[[37, 37, 37], [35, 35, 35], [33, 33, 33], [3...",validation_pic (625).jpg
1,"[[[255, 254, 255], [255, 254, 255], [255, 254,...",validation_pic (275).jpg
2,"[[[9, 3, 13], [5, 0, 9], [3, 0, 7], [5, 0, 9],...",validation_pic (330).jpg
3,"[[[64, 86, 48], [123, 143, 106], [124, 144, 10...",validation_pic (449).jpg
4,"[[[159, 153, 141], [156, 150, 136], [145, 138,...",validation_pic (90).jpg


In [126]:
from skimage.color import rgb2gray
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from skimage.feature import hog
from skimage import data, exposure
import cv2

def ft1(image):
    if len(image.shape) == 2: #if the image is in gray scale
        return np.mean(image)
    return np.mean(image[:,:,0])
# Returns the average of the red-channel pictures for the images
def ft3(image):
    """average of the blue-channel intensity - scalar feature"""
    if len(image.shape) == 2:
        return np.mean(image)
    return np.mean(image[:,:,1])
def ft4(image):
    """average of the green-channel intensity - scalar feature"""
    if len(image.shape) == 2:
        return np.mean(image)
    return np.mean(image[:,:,2])
def ft6(image):
    """Histogram of Oriented Gradients - vector feature"""
    fd, _ = hog(resize(rgb2gray(image)), orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True)
    return fd.reshape(1,-1)
def ft7(image):
    """Mean intensity of image - scalar feature"""
    return np.mean(rgb2gray(image))
def ft11(image):
    """Total ratio of blue over other channels - scalar feature"""
    return ft3(image) / (ft1(image)+ft4(image))
def ft12(image):
    """returns black and white 30x30 image - vector feature"""
    return resize(rgb2gray(image), (30,30)).reshape(1,-1)
def ft14(image):
    """product of ratio of blue and mean image intensity"""
    return np.log(ft11(image)+ft7(image))

In [127]:
def resize(image, target_size=(40,40)):
    """
    resizes image and adds zero padding so that it is 224*224
    updates the bboxes (nparray) to match the new image
    returns the new image as nparray and y_true as a dict 'classes' and 'bboxes' (nparray)
    """
    ih, iw    = target_size
    h,  w = image.shape[0], image.shape[1]

    scale = min(iw/w, ih/h)
    nw, nh  = int(scale * w), int(scale * h)
    image_resized = cv2.resize(image, (nw, nh))
    

    image_paded = np.zeros((ih, iw))
    dw, dh = (iw-nw) // 2, (ih-nh) // 2
    image_paded[dh:nh+dh, dw:nw+dw] = image_resized #the original image is centered
    image_paded = image_paded / 255.
    return image_paded

In [140]:
def feature_frame(df, features = [ft1, ft3, ft6, ft11, ft12, ft14]): #PLUS the Mixture of Gaussians Model
    features_list = []
    for feature in features:
        ft = df.Pictures.apply(feature).values
        if ft[0].shape == ():
            features_list.append(ft[:, np.newaxis])
        else:
            features_list.append(np.concatenate(ft, axis=0))
    new_df = np.concatenate(tuple(features_list), axis=1)
    clf_GM = GaussianMixture(n_components = 20).fit(new_df)
    return np.concatenate((new_df, clf_GM.predict(new_df)[:, np.newaxis]), axis=1)
    #Returns data-frame with all the features now inside, and calculated

In [141]:
features_test = feature_frame(testing_set)

In [144]:
predictions = clf_randomforest.predict(features_test)

In [146]:
testing_set['Predictions'] = predictions

In [150]:
testing_set.drop(columns='Pictures').to_csv('predictions.csv', index = False)